In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pandas as pd
import scipy

plt.close('all')

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/Users/nicholasbachand/Documents/Research/Cascade/city_block_cfd'

'/Users/nicholasbachand/Documents/Research/InteriorModeling'

In [25]:
# resultsTime = "20240311-111046"
# resultsTimes = ["20240406-224437", "20240406-234343", "20240406-235655"]
resultsTimes = ["20240408-235543", "20240409-005549", "20240409-005827"]
outputs = []
inputs = []
for resultsTime in resultsTimes:
    df = pd.read_csv(f'{home_dir}/resultsMC/outputs_{resultsTime}.csv', header=[0, 1], index_col=0)
    outputs.append(df)
    df = pd.read_csv(f'{home_dir}/resultsMC/inputs_{resultsTime}.csv', index_col=0)
    df["group"] = resultsTime
    inputs.append(df)
outputs = pd.concat(outputs, axis = "index", ignore_index=True)
inputs = pd.concat(inputs, axis = "index", ignore_index=True)
display(inputs, outputs)


,floorTempAdjustment,hInterior,hExterior,alphaRoof,material_type,windSpeed,wallRoughness,group
0,-4.151563,2.001253,11.217357,0.843408,Medium,3.322042,1.864905,20240408-235543
1,-4.966065,1.934251,4.409396,0.773652,Medium,1.496870,1.231800,20240408-235543
2,-3.728921,1.542551,9.089012,0.790930,Heavy,3.264173,1.501094,20240408-235543
3,-3.820419,1.088520,2.736810,0.791874,Medium,0.129056,1.120528,20240408-235543
4,-3.568809,2.520289,7.330294,0.676811,Light,2.765833,1.340566,20240408-235543
...,...,...,...,...,...,...,...,...
295,-4.052838,1.860244,6.880376,0.831550,Medium,1.871200,1.832894,20240409-005827
296,-3.854144,2.520817,12.478498,0.643323,Heavy,3.885517,1.851877,20240409-005827
297,-4.722426,1.119643,13.752939,0.802078,Light,5.822008,1.465024,20240409-005827
298,-3.915211,1.960244,10.621074,0.886382,Light,3.767571,1.597425,20240409-005827


0                                                          \
         hVent dVent ceilingMinusFloor outMinusFloor intWallMinusFloor   
0    58.336111   2.0          1.160281      0.750155          0.640894   
1    10.744444   0.0          1.186441      0.782328          0.820390   
2    59.334722   2.0          1.738984      0.552814          0.136090   
3     9.855556   0.0          1.660120      0.649519          0.634497   
4     7.565278   0.0          2.826216      1.145373          0.669306   
..         ...   ...               ...           ...               ...   
295   9.916667   0.0          1.336466      0.746796          0.698319   
296  11.044444   0.0          1.348329      0.410527         -0.035597   
297   8.198611   0.0          2.906424      1.483766          0.966740   
298   9.047222   0.0          2.735276      1.589841          1.143741   
299  10.959722   0.0          2.148487      0.560175         -0.043953   

                               1                                        ...  \
    extWallMinusFloor      hVent dVent ceilingMinusFloor outMinusFloor  ...   
0            1.172592  59.337500   2.0          0.843323     -1.977387  ...   
1            1.002430  11.745833   0.0          0.892176     -1.977816  ...   
2            0.542073  60.336111   2.0          1.447714     -1.818976  ...   
3            0.358288  10.856944   0.0          1.187449     -2.322809  ...   
4            1.444444   8.566667   0.0          2.579830     -0.850702  ...   
..                ...        ...   ...               ...           ...  ...   
295          0.930743  10.918056   0.0          0.971413     -2.603946  ...   
296          0.548604  12.045833   0.0          1.116900     -2.253374  ...   
297          2.289009   9.200000   0.0          2.505803     -1.413190  ...   
298          2.526797  10.048611   0.0          2.230343     -1.980750  ...   
299          0.451494  11.961111   0.0          1.840993     -2.159386  ...   

                  119                                                     120  \
    ceilingMinusFloor outMinusFloor intWallMinusFloor extWallMinusFloor hVent   
0                 NaN           NaN               NaN               NaN   NaN   
1                 NaN           NaN               NaN               NaN   NaN   
2                 NaN           NaN               NaN               NaN   NaN   
3                 NaN           NaN               NaN               NaN   NaN   
4                 NaN           NaN               NaN               NaN   NaN   
..                ...           ...               ...               ...   ...   
295               NaN           NaN               NaN               NaN   NaN   
296               NaN           NaN               NaN               NaN   NaN   
297               NaN           NaN               NaN               NaN   NaN   
298               NaN           NaN               NaN               NaN   NaN   
299               NaN           NaN               NaN               NaN   NaN   

                                                                               
    dVent ceilingMinusFloor outMinusFloor intWallMinusFloor extWallMinusFloor  
0     NaN               NaN           NaN               NaN               NaN  
1     NaN               NaN           NaN               NaN               NaN  
2     NaN               NaN           NaN               NaN               NaN  
3     NaN               NaN           NaN               NaN               NaN  
4     NaN               NaN           NaN               NaN               NaN  
..    ...               ...           ...               ...               ...  
295   NaN               NaN           NaN               NaN               NaN  
296   NaN               NaN           NaN               NaN               NaN  
297   NaN               NaN           NaN               NaN               NaN  
298   NaN               NaN           NaN               NaN               NaN  
299   Na

In [34]:
dfFull = pd.DataFrame()
ventHeaders = outputs.columns.get_level_values(0).unique().values
for header in ventHeaders:
    df = pd.concat([inputs, outputs[header][:]], axis=1)
    df["nVent"] = header
    df["run"] = df.index.values
    dfFull = pd.concat([dfFull, df], axis = "index")
    # break

# dfFull = dfFull.dropna()
dfFull = dfFull.reset_index(drop=True)
# dfFull = dfFull.sort_values(["run", "nVent"])
dfFull
# dfFull["hRelVent"] = dfFull["hVent"].diff()
# dfFull["hRelVent"].fillna(0, inplace=True)
# dfFull["hRelVent"] = dfFull["hRelVent"].clip(lower=0)

,floorTempAdjustment,hInterior,hExterior,alphaRoof,material_type,windSpeed,wallRoughness,group,hVent,dVent,ceilingMinusFloor,outMinusFloor,intWallMinusFloor,extWallMinusFloor,nVent,run
0,-4.151563,2.001253,11.217357,0.843408,Medium,3.322042,1.864905,20240408-235543,58.336111,2.0,1.160281,0.750155,0.640894,1.172592,0,0
1,-4.966065,1.934251,4.409396,0.773652,Medium,1.496870,1.231800,20240408-235543,10.744444,0.0,1.186441,0.782328,0.820390,1.002430,0,1
2,-3.728921,1.542551,9.089012,0.790930,Heavy,3.264173,1.501094,20240408-235543,59.334722,2.0,1.738984,0.552814,0.136090,0.542073,0,2
3,-3.820419,1.088520,2.736810,0.791874,Medium,0.129056,1.120528,20240408-235543,9.855556,0.0,1.660120,0.649519,0.634497,0.358288,0,3
4,-3.568809,2.520289,7.330294,0.676811,Light,2.765833,1.340566,20240408-235543,7.565278,0.0,2.826216,1.145373,0.669306,1.444444,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36295,-4.052838,1.860244,6.880376,0.831550,Medium,1.871200,1.832894,20240409-005827,NaN,NaN,NaN,NaN,NaN,NaN,120,295
36296,-3.854144,2.520817,12.478498,0.643323,Heavy,3.885517,1.851877,20240409-005827,NaN,NaN,NaN,NaN,NaN,NaN,120,296
36297,-4.722426,1.119643,13.752939,0.802078,Light,5.822008,1.465024,20240409-005827,NaN,NaN,NaN,NaN,NaN,NaN,120,297
36298,-3.915211,1.960244,10.621074,0.886382,Light,3.767571,1.597425,20240409-005827,NaN,NaN,NaN,NaN,NaN,NaN,120,298


In [ ]:
dropCondition = dfFull["ceilingMinusFloor"] > 10
dfFull = dfFull[~dropCondition]

In [ ]:
def ventRi(delT, V, H = 3):
    g = 10
    Tref = 288.15
    return g * delT / Tref * H / V**2

dfFull.loc[:, "Ri"] = ventRi(dfFull["ceilingMinusFloor"], dfFull["windSpeed"])
dfFull.loc[:, "logRi"] = np.log(dfFull["Ri"])
dfFull

In [ ]:
qois = ["outMinusFloor", "ceilingMinusFloor", "extWallMinusFloor", "intWallMinusFloor"]
fig = px.histogram(
    dfFull,
    x = qois,
    marginal="box", # or violin, rug
    barmode = "group"
    )
fig.show()

fig = px.line(dfFull, y = qois)
fig.show()


In [ ]:
facet_col_order = ['Light', 'Medium', 'Heavy']
fig = px.scatter(dfFull, x="floorTempAdjustment", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="hInterior", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="alphaRoof", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="dVent", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="hDayVent", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="windSpeed", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="wallRoughness", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
fig = px.scatter(dfFull, x="hExterior", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
fig.show()
# fig = px.scatter(dfFull, x="logRi", y=qois, facet_col="material_type", trendline="ols", category_orders={'material_type': facet_col_order})
# fig.show()

In [ ]:
fig = px.scatter(dfFull, x="windSpeed", y="logRi", symbol="material_type")
fig.show()
fig = px.scatter(dfFull.loc[dfFull["windSpeed"] >= 2], x="windSpeed", y="Ri", symbol="material_type")
fig.show()

In [ ]:
fig = px.scatter_3d(dfFull, x="outMinusFloor", y="ceilingMinusFloor", z="extWallMinusFloor", color="intWallMinusFloor", symbol="material_type", size = "windSpeed")
fig.show()

fig = px.scatter_3d(dfFull, x="outMinusFloor", y="ceilingMinusFloor", z="intWallMinusFloor", color="hDayVent", symbol="material_type", size = "windSpeed")
fig.show()

# fig = px.scatter(dfFull, x="outMinusFloor", y="ceilingMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()
# fig = px.scatter(dfFull, x="outMinusFloor", y="extWallMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()
# fig = px.scatter(dfFull, x="ceilingMinusFloor", y="extWallMinusFloor", color="intWallMinusFloor", symbol="material_type",  size = "windSpeed")
# fig.show()

In [ ]:
plt.figure()
qoiX = dfFull[qois]
U, S, V = np.linalg.svd(qoiX)
plt.plot(S)
display(S)

In [ ]:
qoiX @ V[0:2,:].T

In [ ]:
dfFull['windSpeed']

In [ ]:
fig = px.scatter_3d(qoiX @ V.T, x=0, y=1, z=2, color=3)
fig.show()

fig = px.scatter(qoiX @ V[0:2,:].T, x=0, y=1, color=dfFull["windSpeed"], symbol=dfFull["material_type"])
fig.show()

fig = px.scatter(qoiX @ V[2:,:].T, x=0, y=1, color=dfFull["windSpeed"], symbol=dfFull["material_type"])
fig.show()

In [ ]:
qoiX

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Assuming X contains your 4-dimensional data
# X = [[feature1_1, feature2_1, feature3_1, feature4_1],
#      [feature1_2, feature2_2, feature3_2, feature4_2],
#      ...
#      [feature1_n, feature2_n, feature3_n, feature4_n]]

# Step 2: Choose the number of clusters (k)
k = 3

# Step 3: Apply K-Means Algorithm
kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')

# Step 4: Fit the model
kmeans.fit(qoiX)

# Step 5: Cluster Assignment
labels = kmeans.labels_

# Step 6: Interpret Results
centroids = kmeans.cluster_centers_

# Analyze the clusters and visualize the results as needed


# Plotting the clusters
plt.figure(figsize=(10, 6))

# Plot data points
sns.scatterplot(data=qoiX, x='ceilingMinusFloor', y='extWallMinusFloor', hue=labels, style=dfFull["material_type"], palette='viridis', legend='full', s=50)

# Plot centroids
plt.scatter(centroids[:, 1], centroids[:, 2], marker='x', s=100, c='red', label='Centroids')

# plt.xlabel('ceilingMinusFloor')
# plt.ylabel('outMinusFloor')
plt.title('K-Means Clustering')

plt.legend()
plt.show()
